In [6]:
import random
import pandas as pd


class CampusSpace:

    def __init__(self, buildings_df, hospital_starting_point, grid_size):
        self.buildings_df = buildings_df
        self.hospital_starting_point = hospital_starting_point
        self.grid_size = grid_size
        self.hospitals = set()

    def initialize_hospitals(self, num_hospitals):
        self.hospitals = set()
        # Introduce randomness in initial hospital positions (within a small radius)
        max_delta = 0.001  # Adjust this value to control the random offset radius
        for _ in range(num_hospitals):
            random_offset_lat = random.uniform(-max_delta, max_delta)
            random_offset_lon = random.uniform(-max_delta, max_delta)
            initial_position = (
                self.hospital_starting_point[0] + random_offset_lat,
                self.hospital_starting_point[1] + random_offset_lon,
            )
            self.hospitals.add(tuple(initial_position))

    def available_spaces(self):
        candidates = set(
            (row, col) for row in range(self.grid_size[0]) for col in range(self.grid_size[1])
        )

        # Remove all hospitals
        for hospital in self.hospitals:
            candidates.discard(hospital)
        return candidates

    def hill_climb(self, maximum=None, restarts=5, log=False):
        best_hospitals = None
        best_cost = float('inf')

        for _ in range(restarts):
            # Re-initialize hospitals with random positions at each restart
            self.initialize_hospitals(num_hospitals=1)
            if log:
                print(f"Restart #{_+1} - Initial state: cost", self.get_cost(self.hospitals))

            count = 0
            while maximum is None or count < maximum:
                count += 1
                best_neighbors = []
                best_neighbor_cost = None

                # Consider all hospitals to move
                for hospital in self.hospitals:

                    # Consider all neighbors for that hospital
                    for replacement in self.get_neighbors(hospital):

                        # Generate a neighboring set of hospitals
                        neighbor = self.hospitals.copy()
                        neighbor.remove(hospital)
                        neighbor.add(replacement)

                        # Check if neighbor is best so far
                        cost = self.get_cost(neighbor)
                        if best_neighbor_cost is None or cost < best_neighbor_cost:
                            best_neighbor_cost = cost
                            best_neighbors = [neighbor]
                        elif best_neighbor_cost == cost:
                            best_neighbors.append(neighbor)

                # None of the neighbors are better than the current state
                if best_neighbor_cost >= self.get_cost(self.hospitals):
                    break

                # Move to a highest-valued neighbor
                else:
                    if log:
                        print(f"better neighbor: cost {best_neighbor_cost}")
                    self.hospitals = random.choice(best_neighbors)

            # Update best solution if found a better one
            if self.get_cost(self.hospitals) < best_cost:
                best_hospitals = self.hospitals.copy()
                best_cost = self.get_cost(self.hospitals)

        return best_hospitals

    def get_cost(self, hospitals):
        cost = 0
        for _, building in self.buildings_df.iterrows():
            building_coords = (building['y'], building['x'])
            min_distance = min(
                abs(building_coords[0] - hospital[0]) + abs(building_coords[1] - hospital[1])
                for hospital in hospitals
            )
            cost += min_distance
        return cost

    def get_neighbors(self, hospital):
        lat_step = (self.buildings_df['y'].max() - self.buildings_df['y'].min()) / self.grid_size[0]
        lon_step = (self.buildings_df['x'].max() - self.buildings_df['x'].min()) / self.grid_size[1]

        candidates = [
            (hospital[0] - lat_step, hospital[1]),
            (hospital[0] + lat_step, hospital[1]),
            (hospital[0], hospital[1] - lon_step),
            (hospital[0], hospital[1] + lon_step)
        ]

        neighbors = []
        for r, c in candidates:
            if (r, c) in self.hospitals:
                continue
            if self.buildings_df['y'].min() <= r <= self.buildings_df['y'].max() and self.buildings_df['x'].min() <= c <= self.buildings_df['x'].max():
                neighbors.append((r, c))

        return neighbors


dataset_path = r"C:\Users\MINATI KONWAR\Downloads\latitude and longitude.csv"
campus_data = pd.read_csv(dataset_path)


hospital_starting_point = (23.28540615, 77.27402556)

campus_space = CampusSpace(buildings_df=campus_data, hospital_starting_point=hospital_starting_point, grid_size=(100, 100))

optimal_hospitals = campus_space.hill_climb(log=True)


print("Optimal hospital location:", optimal_hospitals)

Restart #1 - Initial state: cost 0.49549951259299263
better neighbor: cost 0.4889610345969899
better neighbor: cost 0.4824225566009872
better neighbor: cost 0.4758840786049845
better neighbor: cost 0.46959711341699517
better neighbor: cost 0.4634487269330094
better neighbor: cost 0.45747881224100695
better neighbor: cost 0.45177721971301565
better neighbor: cost 0.4460915866730133
better neighbor: cost 0.44040595363301094
better neighbor: cost 0.43506192863303284
better neighbor: cost 0.4299448588970307
better neighbor: cost 0.4248277891610286
better neighbor: cost 0.42000455357705135
better neighbor: cost 0.4158704042010797
better neighbor: cost 0.41189046107307803
better neighbor: cost 0.40811922397708855
better neighbor: cost 0.404423562501087
better neighbor: cost 0.40122097643313026
better neighbor: cost 0.3985681306891564
better neighbor: cost 0.39644802391317313
better neighbor: cost 0.3947160512411898
better neighbor: cost 0.3932299040732019
better neighbor: cost 0.391808495813

In [3]:
import numpy as np
import pandas as pd

data = pd.read_csv(r"C:\Users\MINATI KONWAR\Downloads\latitude and longitude.csv")
campus_bounds = {'min_x': data['x'].min(),'max_x': data['x'].max(),'min_y': data['y'].min(),'max_y': data['y'].max()}


class iiserb():

    def __init__(self, buildings_df, campus_bounds):
        self.buildings_df = buildings_df
        
        self.campus_bounds = campus_bounds

    def euclidean_distance(self, x1, y1, x2, y2):
        return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    def initialize_population(self, pop_size):
        initial_population = []
        for _ in range(pop_size):
            x = np.random.uniform(self.campus_bounds['min_x'], self.campus_bounds['max_x'])
            y = np.random.uniform(self.campus_bounds['min_y'], self.campus_bounds['max_y'])
            initial_population.append([x, y])
        return np.array(initial_population)

    def calculate_fitness(self, hospital_location):
        distances = []
        for _, building in self.buildings_df.iterrows():
            dist = self.euclidean_distance(hospital_location[0], hospital_location[1], building['x'], building['y'])
            distances.append(dist)
        return np.mean(distances)

    def selection_roulette_wheel(self, population, fitness_scores):
        probabilities = 1 / (fitness_scores + 1e-6)
        probabilities /= probabilities.sum()
        selected_indices = np.random.choice(np.arange(len(population)), size=len(population), replace=True, p=probabilities)
        return population[selected_indices]

    def crossover(self, parents, crossover_rate):
        children = []
        for i in range(0, len(parents), 2):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(2)
                child1 = np.concatenate((parents[i][:crossover_point], parents[i+1][crossover_point:]))
                child2 = np.concatenate((parents[i+1][:crossover_point], parents[i][crossover_point:]))
                children.append(child1)
                children.append(child2)
            else:
                children.append(parents[i])
                children.append(parents[i+1])
        return np.array(children)

    def mutation(self, population, mutation_rate):
        for i in range(len(population)):
            if np.random.rand() < mutation_rate:
                population[i] += np.random.normal(0, 0.001, size=2)  
        return population

    def optimize_hospital_location(self, pop_size=100, num_generations=100, crossover_rate=0.8, mutation_rate=0.01):
        population = self.initialize_population(pop_size)
        for gen in range(num_generations):
            fitness_scores = np.zeros(pop_size)
            for i, hospital_location in enumerate(population):
                fitness_scores[i] = self.calculate_fitness(hospital_location)
            population = self.selection_roulette_wheel(population, fitness_scores)
            population = self.crossover(population, crossover_rate)
            population = self.mutation(population, mutation_rate)
        best_index = np.argmin(fitness_scores)
        best_hospital_location = population[best_index]
        return best_hospital_location

iiserb_optimization = iiserb(buildings_df=data, campus_bounds=campus_bounds)
optimal_location = iiserb_optimization.optimize_hospital_location()
print("Optimal hospital location:", optimal_location)

Optimal hospital location: [77.2749789  23.28806948]
